In [12]:
%pip install -U langchain-google-genai
%pip install langchain_community
%pip install pypdf
%pip install faiss-cpu
%pip install langgraph

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/143.7 KB ? eta -:--:--  Downloading langgraph-0.5.0-py3-none-any.whl (143 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 KB 424.6 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 KB 424.6 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/43.8 KB ? eta -:--:--  Downloading langgraph_checkpoint-2.1.0-py3-none-any.whl (43 kB)
     ━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

In [4]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [5]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

embedding_dim = len(embeddings.embed_query("hello world"))
index = faiss.IndexFlatL2(embedding_dim)

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [9]:
from langchain_community.document_loaders import PyPDFLoader

file_path = ("SRS_Doc.pdf")
loader = PyPDFLoader(file_path)
pages = loader.load_and_split()

pages[0]

Document(metadata={'producer': 'Skia/PDF m126 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'SRS Doc', 'source': 'SRS_Doc.pdf', 'total_pages': 130, 'page': 0, 'page_label': '1'}, page_content='Travel AgencyManagementSystem\nSE406:SoftwareRequirementAndSpecificationAnalysis\nS u b m i t t e d \nb y : \nG r o u p \n9 \nR e d o w a n \nB i l l a h \nS a m i \n- \nB S S E 1 4 0 9 \nS e j u t i \nS h a r m i n \nS i d d i q u i \n- \nB S S E 1 4 2 0 \nM d . \nK i b r i a \nH o s s e n \nR o n i \n- \nB S S E 1 4 3 0 \nF a r h a n \nT a u s i f \n- \nB S S E 1 4 4 0 \nR i f a t \nA h m e d \nT u s h e r \n- \nB S S E 1 4 5 1 \nS u b m i t t e d \nt o : \n Dr. KaziMuheymin-Us-Sakib\n(Professor)\nInstituteofInformationTechnologyUniversityofDhaka\n[23-05-24]')

In [ ]:
vector_store.add_documents(pages)

print(f"Successfully embedded {len(pages)} pages directly")
print(f"Vector store now contains {vector_store.index.ntotal} vectors")

Successfully embedded 130 pages directly
Vector store now contains 130 vectors


In [13]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from typing_extensions import List, TypedDict
from langchain_core.documents import Document
from langgraph.graph import StateGraph, START

prompt = hub.pull("rlm/rag-prompt")

# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}

def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}

# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()


/home/kibria/Desktop/coding/Project/RAG_pdf/env/lib/python3.10/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [14]:
# Test the RAG system
result = graph.invoke({"question": "Give me the list of Stakeholders."})
print(result["answer"])

The stakeholders of a Travel Agency are Owners, Members, and Travelers. The list also includes Bus owners, Ship owners, Hotel owners, Resort owners, and Local guides. These stakeholders play a vital role in the entire process.


In [16]:
# Test the RAG system
result = graph.invoke({"question": "Who is the supervisor under whom the SRS is done?"})
print(result["answer"])

I am sorry, but the provided context does not contain information about who the supervisor is for the SRS.


In [18]:
print(graph.invoke({"question": "List of data objects."})['answer'])

The data objects are Variable, User, Itinerary, Album, and Payment. Other data objects include Profile, Username, GoogleAccount, FacebookAccount, Dashboard, and Notification. Additional data objects are Reviews, Admin, HR, and Accountant.
